In [1]:
from Classification import classification_class as classification
from Features_extraction import feature_extraction_class as feature_extraction
from Data_processing.images_processing_class import ImagesProcessing
from utils import perform_classification, best_preprocessing

import pandas as pd
import numpy as np
import cv2
import json

In [2]:
metrics_results_best_methods = {
        "accuracy" : {},
        "f1-score" : {},
        "recall" : {},
        "precision" : {},
        "roc_auc" : {}
        }

name_best_models = {}

In [3]:
data_normal_path = "Data/normal/"
data_pothole_path = "Data/potholes/"

image_process = ImagesProcessing(folder_normal=data_normal_path, folder_potholes=data_pothole_path, img_size=(256, 256))
images = image_process.images
Y = image_process.labels
df_Y = pd.DataFrame(Y)

In [4]:
env_feature_extraction_unprocessed_images = feature_extraction.FeatureExtraction(image_process)

Find best configurations for preprocessing

In [5]:
# TODO: Change this filter with the good association method-filter find in articles

filters = {
    "SIFT": "bilateral",
    "ORB": "bilateral",
    "Harris": "bilateral",
    "EDGE": "bilateral",
}

best_configs = {}

for method_name, filter_name in filters.items():
    best_config, all_results = best_preprocessing(
        image_process, getattr(env_feature_extraction_unprocessed_images, f"method_{method_name}"), filter_name, method_name, n_iter=30
    )
    best_configs[method_name] = best_config

Extracting SIFT Features
Extracting SIFT Features
Extracting SIFT Features
Extracting SIFT Features
Extracting SIFT Features
Extracting SIFT Features
Extracting SIFT Features
Extracting SIFT Features
Extracting SIFT Features
Extracting SIFT Features
Extracting SIFT Features
Extracting SIFT Features
Extracting SIFT Features
Extracting SIFT Features
Extracting SIFT Features
Extracting SIFT Features
Extracting SIFT Features
Extracting SIFT Features
Best preprocessing configuration for SIFT: {'filter': 'bilateral', 'filter_params': (12, 100, 100), 'histogram': 'clahe', 'normalize': False, 'f1-score': 0.804676279657609}
Extracting ORB Features
Extracting ORB Features
Extracting ORB Features
Extracting ORB Features
Extracting ORB Features
Extracting ORB Features
Extracting ORB Features
Extracting ORB Features
Extracting ORB Features
Extracting ORB Features
Extracting ORB Features
Extracting ORB Features
Extracting ORB Features
Extracting ORB Features
Extracting ORB Features
Extracting ORB Fe

In [6]:
with open("best_configs_processing.json", "w") as f:
    json.dump(best_configs, f, indent=4)